In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import sqlite3
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [8]:
conn = sqlite3.connect('cars_test.db')
sql_query = '''

      SELECT
          * 
      FROM
        inventory_test;

'''
result = pd.read_sql_query(sql_query, conn)
result

,title,year,make,model_trim,vehicle_type,model,trim,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,drivetrain,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
0,2021 Acura ILX Sedan w/Premium Package,2021,Acura,ILX w/Premium Package,Sedan,ILX,w/Premium Package,6078,27490,Performance Red Pearl,Black,Automatic,Premium Unleaded I-4 2.4 L/144,FWD,19UDE2F70MA011357,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:05:25.395417
1,2020 Acura ILX Sedan,2020,Acura,ILX Base,Sedan,ILX,Base,30384,22990,Gray,Black,Automatic,Premium Unleaded I-4 2.4 L/144,FWD,19UDE2F34LA004973,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:05:25.395417
2,2020 Acura MDX SH-AWD 7-Passenger,2020,Acura,MDX 3.5L,SUV,MDX,3.5L,6332,37990,Red,Beige,Automatic,Premium Unleaded V-6 3.5 L/212,AWD,5J8YD4H30LL004177,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:05:25.395417
3,2019 Acura MDX SH-AWD w/Technology/A-Spec Pkg,2019,Acura,MDX w/Technology/A-Spec Pkg,SUV,MDX,w/Technology/A-Spec Pkg,34422,37990,Blue Pearl,Red,Automatic,Premium Unleaded V-6 3.5 L/212,AWD,5J8YD4H06KL021182,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:05:25.395417
4,2020 Acura MDX SH-AWD 7-Passenger w/Technology...,2020,Acura,MDX w/Technology Pkg,SUV,MDX,w/Technology Pkg,15703,39990,White,None,Automatic,Premium Unleaded V-6 3.5 L/212,AWD,5J8YD4H56LL033894,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:05:25.395417
5,2020 Acura RDX SH-AWD w/Technology Pkg,2020,Acura,RDX w/Technology Pkg,SUV,RDX,w/Technology Pkg,34116,34490,Black,Black,Automatic,Intercooled Turbo Premium Unleaded I-4 2.0 L/122,AWD,5J8TC2H50LL007082,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:05:25.395417
6,2019 Acura TLX 3.5L SH-AWD w/Technology Pkg,2019,Acura,TLX w/Technology Pkg,Sedan,TLX,w/Technology Pkg,33646,28990,Silver,Black,Automatic,Premium Unleaded V-6 3.5 L/212,AWD,19UUB3F52KA801723,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:05:25.395417
7,2020 Acura TLX 3.5L FWD,2020,Acura,TLX 3.5L V6,Sedan,TLX,3.5L V6,52756,24990,Gray,Black,Automatic,Premium Unleaded V-6 3.5 L/212,FWD,19UUB2F31LA001585,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:05:25.395417
8,2021 Acura TLX FWD w/Technology Package,2021,Acura,TLX w/Technology Package,Sedan,TLX,w/Technology Package,12371,37490,Violet,Black,Automatic,Intercooled Turbo Premium Unleaded I-4 2.0 L/122,FWD,19UUB5F44MA013144,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:05:25.395417
9,2020 Acura TLX 3.5L SH-AWD w/Technology Pkg,2020,Acura,TLX w/Technology Pkg,Sedan,TLX,w/Technology Pkg,38556,30990,Platinum White Pearl,None,Automatic,Premium Unleaded V-6 3.5 L/212,AWD,19UUB3F44LA001580,Stream Auto Outlet,324 W Merrick Rd,11580,Valley Stream,NY,https://www.streamautooutlet.com/inventory?typ...,2022-09-04 03:05:25.395417


In [9]:
print('length: ' , len(result))
print(result.head())

length:  120
                                               title  year   make                   model_trim vehicle_type model                     trim vehicle_mileage  price         exterior_color interior_color transmission                          engine drivetrain                vin     dealership_name dealership_address dealership_zipcode dealership_city dealership_state                                      inventory_url                scraped_date
0             2021 Acura ILX Sedan w/Premium Package  2021  Acura        ILX w/Premium Package        Sedan   ILX        w/Premium Package            6078  27490  Performance Red Pearl          Black    Automatic  Premium Unleaded I-4 2.4 L/144        FWD  19UDE2F70MA011357  Stream Auto Outlet   324 W Merrick Rd              11580   Valley Stream               NY  https://www.streamautooutlet.com/inventory?typ...  2022-09-04 03:05:25.395417
1                               2020 Acura ILX Sedan  2020  Acura                     ILX Base   

In [3]:
conn = sqlite3.connect('cars_test.db')
sql_query = '''

      SELECT
          * 
      FROM
        parsing_errors;

'''
result = pd.read_sql_query(sql_query, conn)
result

,error,dealership,date,url
0,Data Validation,Irwin Automotive Group,2022-09-03 20:43:00.058417,https://www.streamautooutlet.com/inventory?typ...
1,Data Validation,Irwin Automotive Group,2022-09-03 20:43:07.809293,https://www.streamautooutlet.com/inventory?typ...
2,Data Validation,Irwin Automotive Group,2022-09-03 20:43:14.342740,https://www.streamautooutlet.com/inventory?typ...
3,Data Validation,Irwin Automotive Group,2022-09-03 20:43:19.351296,https://www.streamautooutlet.com/inventory?typ...
4,Data Validation,Irwin Automotive Group,2022-09-03 20:43:24.631276,https://www.streamautooutlet.com/inventory?typ...
5,Data Validation,Irwin Automotive Group,2022-09-03 20:43:29.903702,https://www.streamautooutlet.com/inventory?typ...
6,Data Validation,Irwin Automotive Group,2022-09-03 20:43:32.910448,https://www.streamautooutlet.com/inventory?typ...
7,Data Validation,Irwin Automotive Group,2022-09-03 20:59:18.666601,https://www.streamautooutlet.com/inventory?typ...
8,Data Validation,Irwin Automotive Group,2022-09-03 20:59:24.249168,https://www.streamautooutlet.com/inventory?typ...
9,Data Validation,Irwin Automotive Group,2022-09-03 20:59:29.778529,https://www.streamautooutlet.com/inventory?typ...


In [4]:
####### Drop Table

TABLE_NAME = 'inventory_test'
DB_NAME = 'cars_test.db'

#Connecting to sqlite
conn = sqlite3.connect(DB_NAME)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Doping EMPLOYEE table if already exists
cursor.execute(f"DROP TABLE {TABLE_NAME}")
print("Table dropped... ")

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()


Table dropped... 
